# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

## Fire up graphlab create

In [1]:
# python standard library
import os
from math import log, sqrt

# third party
import graphlab
import numpy

# this code
import machine_learning

In [2]:
data_path = os.path.join(machine_learning.__path__[0], machine_learning.large_data_path, 'kc_house_data.gl/')

## Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame(data_path)

[INFO] GraphLab Server Version: 1.7.1


[INFO] Start server at: ipc:///tmp/graphlab_server-11729 - Server binary: /home/charon/.virtualenvs/machinelearning/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1451775741.log


[INFO] 1451775741 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /home/charon/.virtualenvs/machinelearning/local/lib/python2.7/site-packages/certifi/cacert.pem
1451775741 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to necromuralist@gmail.com and will expire on October 20, 2016. For commercial licensing options, visit https://dato.com/buy/.



## Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

## Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.381509     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 1.416980     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [12]:
coefficients = model_all.get('coefficients')
coefficients.print_rows(num_rows=18)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274873.05595 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 8468.53108691 |
|   sqft_living    |  None | 24.4207209824 |
| sqft_living_sqrt |  None | 350.060553386 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 842.068034898 |
|    sqft_above    |  None | 20.0247224171 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION 1***:
According to this list of weights, which of the features have been chosen? 

In [19]:
all(value>=0 for value in coefficients['value'])

True

The previous check tells us that there's no negative coefficient values so we can use a one-sided inequality when looking for non-zero weights.

In [13]:
non_zeros = ((index, value) for index, value in enumerate(coefficents['value']) if value > 0)
for index, value in non_zeros:
    print("{0}: {1:.2f}".format(coefficents['name'][index],
                                    value))

(intercept): 274873.06
bathrooms: 8468.53
sqft_living: 24.42
sqft_living_sqrt: 350.06
grade: 842.07
sqft_above: 20.02


bathrooms, sqft_living, sqft_living_sqrt

## Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [14]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [15]:
rss_penalty = []
models_penalties = ((graphlab.linear_regression.create(training, target='price', features=all_features,
                                                       validation_set=None, l2_penalty=0, l1_penalty=l1_penalty,
                                                       verbose=False), l1_penalty) for l1_penalty in numpy.logspace(1, 10, num=19))
models_predictions_penalties = ((model, model.predict(validation), l1_penalty) for model, l1_penalty in models_penalties)
rss_penalty_model = [(((prediction - validation['price'])**2).sum(), l1_penalty, model)
                     for (model, prediction, l1_penalty) in models_predictions_penalties]

*** QUIZ QUESTIONS 2 ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [17]:
best = min(rss_penalty_model)
print("Best l1_penalty: {0}".format(best[1]))
print('lowest RSS: {0}'.format(best[0]))

Best l1_penalty: 10.0
lowest RSS: 6.25766285142e+14


In [18]:
model = best[2]
prediction = model.predict(testing)
rss = ((prediction - testing['price'])**2).sum()
print('RSS of the model on the testing data: {0:.2f}'.format(rss))

RSS of the model on the testing data: 156983602381664.19


***QUIZ QUESTION 3***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [20]:
coefficents = best[2].get('coefficients')

In [21]:
all(value >= 0 for value in coefficients['value'])

True

In [24]:
non_zeros = ((coefficients['name'][index], value) for index, value in enumerate(coefficients['value']) if value > 0)
for index, (name, value) in enumerate(non_zeros):
    print("({2}) {0}: {1:.2f}".format(name, value, index))

(0) (intercept): 274873.06
(1) bathrooms: 8468.53
(2) sqft_living: 24.42
(3) sqft_living_sqrt: 350.06
(4) grade: 842.07
(5) sqft_above: 20.02


In [12]:
worst_model = max(rss_penalty_model)
print("worst l1_penalty: {0}".format(worst_model[1]))
print('highest RSS: {0}'.format(worst_model[0]))
print(worst_model[2].get('coefficients'))

worst l1_penalty: 10000000000.0
highest RSS: 1.22915716064e+15
+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 564482.136844 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None |      0.0      |
|   sqft_living    |  None |      0.0      |
| sqft_living_sqrt |  None |      0.0      |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


## Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [25]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [26]:
l1_penalty_values = numpy.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [27]:
nonzeros_penalty = []
more_nonzeros_penalty = []
fewer_nonzeros_penalty = []
exact_nonzeros_penalty = []
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, l1_penalty=l1_penalty, l2_penalty=0, verbose=False)
    non_zeros = model['coefficients']['value'].nnz()
    nonzeros_penalty.append((non_zeros, l1_penalty))
    if non_zeros < max_nonzeros:
        fewer_nonzeros_penalty.append((non_zeros, l1_penalty))
    elif non_zeros > max_nonzeros:
        more_nonzeros_penalty.append((non_zeros, l1_penalty))
    else:
        exact_nonzeros_penalty.append((non_zeros, l1_penalty))

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [28]:
print(nonzeros_penalty)

[(18, 100000000.0), (18, 127427498.57031322), (18, 162377673.91887242), (18, 206913808.11147901), (17, 263665089.87303555), (17, 335981828.62837881), (17, 428133239.8719396), (17, 545559478.11685145), (17, 695192796.17755914), (16, 885866790.41008317), (15, 1128837891.6846883), (15, 1438449888.2876658), (13, 1832980710.8324375), (12, 2335721469.0901213), (10, 2976351441.6313133), (6, 3792690190.7322536), (5, 4832930238.5717525), (3, 6158482110.6602545), (1, 7847599703.5146227), (1, 10000000000.0)]


In [29]:
print("Max Nonzeros: {0}".format(max_nonzeros))

opposite_min_max = {min: max,
                    max: min}

def get_boundary(min_or_max_nonzeros, source):
    """
    :param:
     - `min_or_max_nonzeros`: function to find boundary based on non-zeros
     - `source`: iterable containing potential boundaries
    :return: tuple in source that has min or max penalt
    """    
    boundary_nonzeros = min_or_max_nonzeros(source)[0]
    boundary_nonzeros_penalty = (n_p for n_p in nonzeros_penalty if n_p[0] == boundary_nonzeros)
    return opposite_min_max[min_or_max_nonzeros]((penalty, zeros) for (zeros, penalty) in boundary_nonzeros_penalty)

lower_bound = get_boundary(min, more_nonzeros_penalty)
upper_bound = get_boundary(max, fewer_nonzeros_penalty)
print("Lower Bound: l1 penalty={0} zeros={1}".format(lower_bound[0], lower_bound[1]))
print("Upper Bound: l1 penalty={0} zeros={1}".format(upper_bound[0], upper_bound[1]))

Max Nonzeros: 7
Lower Bound: l1 penalty=2976351441.63 zeros=10
Upper Bound: l1 penalty=3792690190.73 zeros=6


In [31]:
l1_penalty_min = lower_bound[0]
l1_penalty_max = upper_bound[0]

***QUIZ QUESTIONS 4***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

In [32]:
print(l1_penalty_min)
print(l1_penalty_max)

2976351441.63
3792690190.73


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [33]:
l1_penalty_values = numpy.linspace(l1_penalty_min, l1_penalty_max, 20)

In [34]:
rss_penalty_nonzeros_model = []
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training,
                                              target='price',
                                              features=all_features,
                                              l1_penalty=l1_penalty,
                                              l2_penalty=0,
                                              validation_set=None,
                                              verbose=False)
    predictions = model.predict(validation)
    non_zeros = model['coefficients']['value'].nnz()
    residuals = predictions - validation['price']
    rss_penalty_nonzeros_model.append(((residuals**2).sum(), l1_penalty, non_zeros, model))
    

In [35]:
best = min(((rss, penalty, non_zeros, model) for rss, penalty, non_zeros, model in rss_penalty_nonzeros_model if non_zeros == max_nonzeros))
print("RSS: {0}, L1 Penalty: {1} Non-Zeros: {2}".format(*best[0:3]))
assert best[2] == max_nonzeros

        

RSS: 1.04693748875e+15, L1 Penalty: 3448968612.16 Non-Zeros: 7


***QUIZ QUESTIONS 5 ***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [61]:
coefficients = best[3].get('coefficients')
print(coefficients)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 222253.192544 |
|     bedrooms     |  None | 661.722717782 |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 15873.9572593 |
|   sqft_living    |  None | 32.4102214513 |
| sqft_living_sqrt |  None | 690.114773313 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [37]:
for index, value in enumerate(coefficients['value']):
    if value != 0:
        print("'{0}'\t{1:.2f}".format(coefficients['name'][index],
                                                                  coefficients['value'][index]))

'(intercept)'	274873.06
'bathrooms'	8468.53
'sqft_living'	24.42
'sqft_living_sqrt'	350.06
'grade'	842.07
'sqft_above'	20.02